<a href="https://colab.research.google.com/github/sohailkhanmarwat/Advanced-Machine-Learning-National-Research-University-Higher-School-of-Economics/blob/master/train_indian_pines_dataset_cnn_spp_rnn_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! git clone https://github.com/sohailkhanmarwat/datasets.git

fatal: destination path 'datasets' already exists and is not an empty directory.


In [0]:
#from google.colab import files
#uploaded = files.upload()

In [0]:
import numpy as np
import scipy
import os
import keras
import keras.layers
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten,  Activation, Input, GRU, TimeDistributed, AveragePooling2D, MaxPooling2D
from keras.layers import Bidirectional, Conv2D, MaxPooling2D, LeakyReLU, BatchNormalization, ReLU, LSTM, RNN, Softmax, SimpleRNN
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, Adamax
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.regularizers import L1L2,l1_l2,l1,l2
K.set_image_dim_ordering('th')
from keras.utils import np_utils
#from sklearn.cross_validation import StratifiedKFold
from SpatialPyramidPooling import SpatialPyramidPooling
import matplotlib.pyplot as plt
from keras.initializers import glorot_uniform
from keras.layers import SimpleRNN


In [0]:
numPCAcomponents = 30
# Patches windows size
windowSize = 7
# The proportion of Test sets
testRatio = 0.75

In [0]:
cd datasets/

[Errno 2] No such file or directory: 'datasets/'
/content/datasets


In [0]:
!pwd

/content/datasets


In [0]:
X_train = np.load("./predata/XtrainWindowSize" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio)  + ".npy")
y_train = np.load("./predata/ytrainWindowSize" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")
X_test = np.load("./predata/XtestWindowSize" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")
y_test = np.load("./predata/ytestWindowSize" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")


In [0]:
# Reshape data into (numberofsumples, channels, height, width)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[3], 
                               X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[3], 
                             X_test.shape[1], X_test.shape[2]))

# convert class labels to on-hot encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Define the input shape 
input_shape= X_train[0].shape
print(input_shape)

# number of filters
C1 = 3*numPCAcomponents

(30, 7, 7)


In [0]:
# First, let's define a RNN Cell, as a layer subclass.

class MinimalRNNCell(keras.layers.Layer):

    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        super(MinimalRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='uniform',
            name='recurrent_kernel')
        self.built = True

    def call(self, inputs, states):
        prev_output = states[0]
        h = K.dot(inputs, self.kernel)
        output = h + K.dot(prev_output, self.recurrent_kernel)
        return output, [output]
# Let's use this cell in a RNN layer:

#cell = MinimalRNNCell(32)
#x = keras.Input((None, 5))
#layer = RNN(cell)
#y = layer(x)

# Here's how to use the cell to build a stacked RNN:

#cells = [MinimalRNNCell(32), MinimalRNNCell(64)]
#x = keras.Input((None, 5))
#layer = RNN(cells)
#y = layer(x)

In [0]:
def obtain_datagen(datagen, train_path, h5=True):
	return datagen.flow_from_directory(
				train_path,
				target_size=(windowSize,windowSize),
				batch_size=batch_size,
				class_mode='categorical') 	
    
def createCNNRNNModel():
  
    input_dim = (30, windowSize, windowSize)

    num_timesteps = 1
    input_lstm_shape=(num_timesteps,) + input_dim
    #model_input = Input(shape=input_lstm_shape, name='seq_input')
    print('input_lstm_shape',input_lstm_shape)
    model = Sequential()
    model.add(TimeDistributed(Conv2D(C1, (3, 3), padding='same', input_shape=input_shape, kernel_regularizer=l2(0.001)), input_shape =input_lstm_shape, name="TD1"))
    model.add(TimeDistributed(LeakyReLU(0.1), name="TD2"))
    model.add(TimeDistributed(BatchNormalization(momentum=0.9), name="TD4"))
    model.add(TimeDistributed(Dropout(0.5), name="TD3"))    
    
    #model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),name='MaxPooling2D')))
    
    model.add(TimeDistributed(Conv2D(3*C1, (3, 3)), name="TD5"))
    model.add(TimeDistributed(BatchNormalization(momentum=0.9), name="TD7"))
    model.add(TimeDistributed(LeakyReLU(0.1), name="TD6"))    
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(SpatialPyramidPooling([1, 2, 3, 5]), name="TD8"))
    
    #model.add(Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001))))
    #model.add(TimeDistributed(LeakyReLU(0.1), name="TD10"))
    #model.add(TimeDistributed(BatchNormalization(momentum=0.9), name="TD111"))
    #model.add(TimeDistributed(Dropout(0.5), name="TD122"))
    #model.add(Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001))))
    #model.add(TimeDistributed(LeakyReLU(0.1), name="TD100"))
    #model.add(TimeDistributed(BatchNormalization(momentum=0.9), name="TD11"))
    #model.add(TimeDistributed(Dropout(0.5), name="TD12"))
    #rnnCell = MinimalRNNCell(64)
    cells = [MinimalRNNCell(32), MinimalRNNCell(64), MinimalRNNCell(128)]
    #layer = RNN(rnnCell, return_sequences=True)
    model.add(RNN(cells, return_sequences=True))
    #model.add(TimeDistributed(SimpleRNN(300, return_sequences=True, regularizer=l2(0.001))))
            
    
    #rnn = SimpleRNN(
    #            300,
    #            return_sequences=True,
    #            dropout=self.dropout,
    #            recurrent_dropout=self.recurrent_dropout)(char_emb)
    #        dropped = Dropout(0.5)(rnn)
    #        mot = MeanOverTime(mask_zero=True)(dropped)
    #        densed = Dense(self.num_outputs, name='dense')(mot)
    #        output = Activation('sigmoid')(densed)

    #x = GRU(256, kernel_regularizer=l2(0.001))(x)

    #x = SimpleRNN(256, return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01),
    #                      activity_regularizer=l2(0.01), kernel_constraint='max_norm',
    #                      bias_constraint='max_norm', recurrent_constraint='max_norm')(x)

    
    model.add(Dense(128, name="TD13"))
    model.add(LeakyReLU(0.1, name="TD16"))
    model.add(BatchNormalization(momentum=0.9, name="TD17"))
    model.add(Dropout(0.5, name="TD18"))
    model.add(Dense(16, name="TD19"))
    model.add(Softmax())
    
    return model
model = createCNNRNNModel()
model.summary()

In [0]:
def createModel():
#    model = Sequential()
    # define CNN model
    '''
    Load the output of the CNN
    '''
    #cnn_model = load_model(os.path.join('model_cnn5', 'checkpoint0.058_best.hdf5'))
    
    cnn_model = Sequential()

    cnn_model.add(Conv2D(C1, (3, 3), padding='same', input_shape=input_shape, kernel_regularizer=l2(0.001)))
    cnn_model.add(LeakyReLU(0.1))
    cnn_model.add(BatchNormalization(momentum=0.9))
    cnn_model.add(Dropout(0.3))
    cnn_model.add(Conv2D(3*C1, (3, 3), padding='same'))
    cnn_model.add(LeakyReLU(0.1))
    cnn_model.add(BatchNormalization(momentum=0.9))
    cnn_model.add(Dropout(0.3))
    
    cnn_model.add(SpatialPyramidPooling([1, 2, 4, 5, 7]))

    #model.add(Flatten())
    cnn_model.add(Dense(9*numPCAcomponents))
    cnn_model.add(LeakyReLU(0.1))
    cnn_model.add(BatchNormalization(momentum=0.9))
    cnn_model.add(Dropout(0.3))
    cnn_model.add(Dense(6*numPCAcomponents))
    cnn_model.add(LeakyReLU(0.1))
    cnn_model.add(Dropout(0.3))
    cnn_model.add(Dense(16, activation='softmax'))
    
    return cnn_model
  


In [0]:
#for layer in cnn_model.layers:
#	layer.trainable = False	

Xtrain = np.reshape(X_train,(-1, 1, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
ytrain = np.reshape(y_train,(-1, 1, y_train.shape[1]))

Xtest = np.reshape(X_test, (-1, 1, X_test.shape[1], X_test.shape[2], X_test.shape[3]))
ytest = np.reshape(y_test, (-1, 1, y_test.shape[1]))

In [0]:
optimizer = ['SGD', 'RMSprop']
#l_rates = [.001,.01,.0003,.003,.03,.3]
l_rates = [.05,.03]
#l_rates = [.03,.3]

for lr in l_rates:
  
  l_rate = lr
  print("Learning Rate: " +str(lr))
  #l_rate = lr #round(i,4) # loss: 0.0030 - acc: 0.9990 - val_loss: 0.1607 - val_acc: 0.9629
  reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=25, min_lr=0.000001, verbose=1)
  checkpointer = ModelCheckpoint(filepath="checkpoint %f"%(l_rate)+"_lstm.hdf5", verbose=1, save_best_only=True, mode='max')
  sgd = SGD(lr=l_rate, decay=1e-6, momentum=0.9, nesterov=True)
  rmsprop = RMSprop(lr=l_rate,decay=1e-6, rho=0.9, epsilon=1e-08)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

  # Start to train model 
  history = model.fit(Xtrain, ytrain, 
              batch_size=32, 
              epochs=200, 
              verbose=1, 
              validation_data=(Xtest, ytest),
              callbacks=[reduce_lr, checkpointer],
              shuffle=True)
  #ii += 1
  f = open("train_cnn5_ssp_lstm4.txt", "a+")
  f.write("Learning Rate: %1.6f Accuracy: %2.6f Value Accuracy: %2.6f\r\n"%(history.history["lr"][0],max(history.history["acc"])*100,max(history.history['val_acc'])*100))
  f.close()


  # summarize history for accuracy
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.grid(True)
  plt.legend(['train', 'test'], loc='upper left') 
  plt.savefig("model_accuracy_lstm4_"+str(l_rate)+".svg")
  plt.show()

  # summarize history for loss 
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.grid(True)
  plt.legend(['train', 'test'], loc='upper left') 
  plt.savefig("model_loss_lstm4_"+str(l_rate)+".svg")
  plt.show()

  import h5py
  from keras.models import load_model
  model.save('HSI_model_epochs100_ssp_lstm_'+str(l_rate)+'.h5')

# using plot_model module to save the model figure
from keras.utils import plot_model
plot_model(model, to_file='model_ssp_lstm4.png', show_shapes=True)
print(history.history.keys())

# show the model figure

get_ipython().run_line_magic('matplotlib', 'inline')
model_img = plt.imread('model_ssp_lstm4.png')
plt.imshow(model_img, shape=(7, 7))
plt.show()


